In [2]:
import bioio
import tensorflow as tf

/home/marc/.miniconda/envs/bioio/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset = bioio.load_biospec('biospec.yml')
dataset

In [ ]:
x = next(iter(dataset))
x

In [ ]:
bioio.tf.dataset_to_tfrecord(dataset, 'test.tfrecord', encoding='bytes')

In [ ]:
tf.sparse.from_dense(x['outputs']['SomeTask']['total'])

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
tfds.dataset_builders.store_as_tfds_dataset(name='ENCODE_TIA1-HepG2', version='0.0.1', split_datasets = {'train': dataset}, features=features, data_dir='test_tfds')

In [ ]:
tfds.dataset_builders.TfDataBuilder(name='ENCODE_TIA1-HepG2', version='0.0.1', split_datasets = {'train': dataset}, data_dir='test_tfds')

In [3]:
dataset = bioio.tf.load_tfrecord('test.tfrecord')
print(dataset)

x = next(iter(dataset))
x

# def to_sparse_if_dense(x):
#     if isinstance(x, tf.SparseTensor):
#         return tf.sparse.to_dense(x)
#     else:
#         return x

# dataset = dataset.map(lambda x: tf.nest.map_structure(to_sparse_if_dense, x))
# print(dataset)

# x = next(iter(dataset))
# x

<_PrefetchDataset element_spec={'outputs': {'SomeTask': {'control': SparseTensorSpec(TensorShape(None), tf.float32), 'total': SparseTensorSpec(TensorShape(None), tf.float32)}}}>


{'outputs': {'SomeTask': {'control': SparseTensor(indices=tf.Tensor([], shape=(0, 1), dtype=int64), values=tf.Tensor([], shape=(0,), dtype=float32), dense_shape=tf.Tensor([201], shape=(1,), dtype=int64)),
   'total': SparseTensor(indices=tf.Tensor(
   [[  0]
    [  1]
    [  2]
    [  3]
    [  4]
    [  5]
    [  6]
    [  7]
    [  8]
    [  9]
    [ 10]
    [ 11]
    [ 12]
    [ 13]
    [ 14]
    [ 16]
    [ 17]
    [ 24]
    [ 92]
    [ 99]
    [100]
    [101]
    [102]
    [103]
    [104]
    [105]
    [106]
    [107]
    [108]
    [109]
    [110]
    [111]
    [112]
    [113]
    [114]
    [115]
    [116]
    [117]
    [118]
    [119]
    [120]
    [121]
    [122]
    [123]
    [124]
    [125]
    [128]
    [130]
    [131]
    [138]
    [139]
    [140]
    [141]
    [142]
    [143]
    [144]
    [145]
    [146]
    [147]
    [148]
    [149]
    [152]
    [153]
    [154]
    [155]
    [156]
    [157]
    [158]
    [159]
    [160]
    [163]
    [165]
    [166]
    [167]
    [168]
 

In [ ]:
tf.sparse.to_dense(x['outputs']['SomeTask']['total'])

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow_datasets.core.proto import feature_pb2
from tensorflow_datasets.core.features import feature as feature_lib

class SparseTensor(tfds.features.FeatureConnector):
    def __init__(self, dtype, **kwargs):
        self._dtype = dtype
        self._doc = tfds.features.Documentation()

        self._st_feature_dict = tfds.features.FeaturesDict(
            {
                'indices': tfds.features.Tensor(shape=(None, None), dtype=tf.int64, encoding='bytes'),
                'values': tfds.features.Tensor(shape=(None, ), dtype=self._dtype),
                'dense_shape': tfds.features.Tensor(shape=(None, ), dtype=tf.int64),
            },
        )

    def _cast_to_numpy(self, x):
        if isinstance(x, tf.Tensor):
            x = x.numpy()
        return x

    def _sparse_to_dict(self, example_data):
        return {
            'indices': self._cast_to_numpy(example_data.indices),
            'values': self._cast_to_numpy(example_data.values),
            'dense_shape': self._cast_to_numpy(example_data.dense_shape)
        }

    def _dict_to_sparse(self, example_data_dict):
        return tf.SparseTensor(
            indices = example_data_dict['indices'],
            values = example_data_dict['values'],
            dense_shape = example_data_dict['dense_shape'],
        )

    def get_serialized_info(self):
        return self._st_feature_dict.get_serialized_info()

    def get_tensor_info(self):
        return None
    
    # def to_json_content(self):
    #     return self._st_feature_dict.to_json_content()

    def to_json_content(self) -> feature_pb2.TensorFeature:
        return feature_pb2.TensorFeature(
            dtype=feature_lib.dtype_to_str(self._dtype),
            # encoding=self._encoding.value,
        )

    @classmethod
    def from_json_content(self, value):
        return SparseTensor(dtype=feature_lib.dtype_from_str(value.dtype))

    def encode_example(self, example: tf.SparseTensor):
        example_as_dict = self._sparse_to_dict(example)
        return self._st_feature_dict.encode_example(example_as_dict)

    def decode_example(self, encoded_example: bytes):
        example_as_dict = self._st_feature_dict.decode_example(encoded_example)
        return self._dict_to_sparse(example_as_dict)

stf = SparseTensor(tf.float32)
stf.to_json()

In [ ]:
# %%
def features_to_json_file(features, filepath, indent=2):
    with open(filepath, 'w') as json_file:
        print(json.dumps(features.to_json(), indent=indent), file=json_file)

# %%
def features_from_json_file(filepath):
    with open(filepath) as json_file:
        features = tfds.features.FeaturesDict.from_json(json.load(json_file))
    return features

st_feature = tfds.features.FeaturesDict({
    'example': SparseTensor(dtype=tf.int32)
})

tfds.features.FeaturesDict.from_json(st_feature.to_json())